In [1]:
!nvidia-smi

Tue May  7 09:14:18 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 536.67                 Driver Version: 536.67       CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                     TCC/WDDM  | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti   WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   46C    P8              43W / 350W |    591MiB / 12288MiB |      6%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
import torch
from torch import nn
torch.device("cpu"),torch.device("cuda"),torch.device("cuda:1")

(device(type='cpu'), device(type='cuda'), device(type='cuda', index=1))

In [4]:
torch.cuda.device_count()

1

In [6]:
def try_gpu(i=0):
    if torch.cuda.device_count() >= i+1:
        return torch.device(f"cuda:{i}")
    return torch.device("cpu")

def try_all_gpus():
    devices = [torch.device(f'cuda:{i}') for i in range(torch.cuda.device_count())]
    return devices if devices else [torch.device("cpu")]
try_gpu(),try_gpu(10),try_all_gpus()

(device(type='cuda', index=0),
 device(type='cpu'),
 [device(type='cuda', index=0)])

In [7]:
#5.6.2. 张量与GPU
x = torch.tensor([1,2,3])
x.device

device(type='cpu')

In [8]:
#5.6.2.1. 存储在GPU上
x = torch.ones(2,3,device=try_gpu())
x

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')

In [10]:
#5.6.2.2. 复制
z = x.cuda(0)
print(x)
print(z)

tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')
tensor([[1., 1., 1.],
        [1., 1., 1.]], device='cuda:0')


In [12]:
z + x

tensor([[2., 2., 2.],
        [2., 2., 2.]], device='cuda:0')

In [13]:
z.cuda(0) is z

True

In [14]:
#5.6.3. 神经网络与GPU
net = nn.Sequential(nn.Linear(3,1))
net = net.to(device=try_gpu())

In [15]:
net(x)

tensor([[0.6327],
        [0.6327]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [16]:
net[0].weight.data.device

device(type='cuda', index=0)

In [17]:
import time
import torch

def try_gpu(i=0):  #@save
    """如果存在，则返回gpu(i)，否则返回cpu()"""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

startTime1=time.time()
for i in range(100):
    A = torch.ones(500,500)
    B = torch.ones(500,500)
    C = torch.matmul(A,B)
endTime1=time.time()

startTime2=time.time()
for i in range(100):
    A = torch.ones(500,500,device=try_gpu())
    B = torch.ones(500,500,device=try_gpu())
    C = torch.matmul(A,B)
endTime2=time.time()

print('cpu计算总时长:', round((endTime1 - startTime1)*1000, 2),'ms')
print('gpu计算总时长:', round((endTime2 - startTime2)*1000, 2),'ms')

cpu计算总时长: 151.0 ms
gpu计算总时长: 4.03 ms
